In [1]:
import numpy as np
from pycomposer.composition_controler import CompositionControler
from pycomposer.inferablepitch.rtrbm_inferer import RTRBMInferer
from pycomposer.inferableconsonance.q_consonance import QConsonance
from pycomposer.midi_vectorlizer import MidiVectorlizer

In [12]:
midi_vectorlizer = MidiVectorlizer()
tone_df = midi_vectorlizer.extract("../../furudokei.mid")
tone_df = tone_df.sort_values(by=["start", "end"])
tone_df.head()

,program,start,end,pitch,velocity,duration
173,48,1.8,2.4,62,95,0.6
174,48,2.4,3.0,67,105,0.6
317,0,2.4,3.6,55,90,1.2
318,0,2.4,3.6,59,100,1.2
175,48,3.0,3.3,66,100,0.3


In [3]:
q_consonance = QConsonance()
q_consonance.initialize()
for pitch in tone_df.pitch.values.tolist():
    q_consonance.learn(state_key=pitch, limit=1)

In [4]:
inferable_pitch = RTRBMInferer(
    learning_rate=0.00001,
    hidden_n=100,
    hidden_binary_flag=True,
    inferancing_training_count=1,
    r_batch_size=200
)

In [5]:
inferable_pitch.learn(tone_df=tone_df, training_count=1, batch_size=200)

In [6]:
composition_controler = CompositionControler(
    resolution=960,
    initial_tempo=120,
    chord_instrument_num=33,
    melody_instrument_num=1
)

In [7]:
chord_melody_list = composition_controler.create_chord_melody_list(
    octave=4, 
    first_chord="I",
    length=4
)

In [8]:
#composition_controler.reset()
composition_controler.match_melody_to_chords(
    inferable_pitch,
    q_consonance,
    chord_melody_list,
    measure_n=4,
    start_measure_n=0,
    beat_n=4,
    metronome_time=100,
    chord_velocity_range=(50, 60),
    melody_velocity_range=(90, 110)
)

In [9]:
composition_controler.save("test.mid")

In [11]:
#midi_vectorlizer.extract("test.mid")